In [1]:
from functools import partial

from cdp_data import CDPInstances, datasets
import pandas as pd

# How to open and convert each transcript
def _process_session(row: pd.Series, extra_meta: dict) -> dict:
    # Read transcript CSV
    transcript = pd.read_csv(row.transcript_as_csv_path)
    
    return {
        "text": " ".join(transcript.text.astype(str)),
        "meta": {
            "event_id": row.event.id,
            "session_id": row.id,
            **extra_meta,
        },
    }

# Get sessions
all_sessions = []
for city in [
    CDPInstances.Seattle,
    CDPInstances.Oakland,
    CDPInstances.Asheville,
    CDPInstances.Boston,
    CDPInstances.Milwaukee,
]:
    print(f"Working on council: {city}")
    # Get all sessions for city
    df = datasets.get_session_dataset(
        city,
        start_datetime="2020-01-01",
        end_datetime="2023-09-01",
        sample=10,
        store_transcript=True,
        store_transcript_as_csv=True,
        raise_on_error=False,
    )

    # Create partial with muni name
    process_session = partial(_process_session, extra_meta={"infrastructure": city})

    # Process all rows in dataset to get full text to a single column
    all_sessions.extend(
        list(df.apply(process_session, axis=1))
    )

# Store to single dataframe
processed_sessions = pd.DataFrame(all_sessions)

# Store processed dataset to JSONL
processed_sessions.to_json("public-comment-seg-multi-city.jsonl", orient="records", lines=True)

/home/eva/miniforge-pypy3/envs/minutes-item-seg/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Working on council: cdp-seattle-21723dcf


Converting transcripts: 100%|██████████| 10/10 [00:01<00:00,  5.03it/s]


Working on council: cdp-oakland-ba81c097


Converting transcripts: 100%|██████████| 10/10 [00:02<00:00,  4.01it/s]


...

Converting transcripts: 100%|██████████| 10/10 [00:02<00:00,  4.15it/s]


Working on council: cdp-boston-c384047b


Converting transcripts: 100%|██████████| 10/10 [00:02<00:00,  3.70it/s]


Working on council: cdp-milwaukee-9f60e352


Converting transcripts: 100%|██████████| 10/10 [00:03<00:00,  3.27it/s]
